In [ ]:
!kill -9 -1

In [ ]:
import os
os.chdir('/')
!rm -rf /content/STUDY

In [ ]:
# Clone the repository
!git clone https://github.com/will-r-smith/STUDY.git /content/STUDY

# Navigate to the repository directory
import os
os.chdir('/content/SELF')

In [ ]:
!python run.py --test fine_tune --model roberta --intervention lr --lname fc_in --lnum 11 --rate 0.9 --prop_data 0.005 --dataset counterfact --batch_size 32 --num_epochs 10 --learning_rate 0.0001